# Entrevista Cajero (WatsonX)

In [1]:
import os
import json
import requests
import re
import time
import random

## Selección Modelo LLM

In [2]:
# Pregunta el modelo a ejecutar
select_model = input ("\nQué modelo vas a ejecutar? Ingresá M (Mixtral-8x7b) o ML (Mistral Large) o G (GPT3.5):  ")


Qué modelo vas a ejecutar? Ingresá M (Mixtral-8x7b) o ML (Mistral Large) o G (GPT3.5):   M


In [26]:
if select_model.lower() == 'm':
    model = "mistralai/mixtral-8x7b-instruct-v01"
elif select_model.lower() == 'ml':
    model = "mistralai/mistral-large"
else:
    model = "gpt-3.5-turbo"
    try:
       import openai
    except:
        !pip install openai
        import openai
    openai.api_key = '' #os.getenv('OPENAI_API_KEY')
print("\n Se ejecutará el modelo: '"+model+"'\n")


 Se ejecutará el modelo: 'mistralai/mixtral-8x7b-instruct-v01'



In [27]:
#config Watsonx.ai environment # NO TOCAR
#load_dotenv()
api_key = "" #os.getenv("API_KEY", None)
ibm_cloud_url = "" #os.getenv("IBM_CLOUD_URL", None)
project_id = "" #os.getenv("PROJECT_ID", None)
creds = {
    "url": ibm_cloud_url,
    "apikey": api_key 
}
token = ''

# propiedades
max_tokens_limit = 8192
max_tokens = 8192
temperature = 0.7
top_p = 1.0
random_seed = random.randrange(1,99)
total_tokens_used = 0
response_seconds = 0.0
total_response_seconds = 0.0
avg_response_seconds = 0.0

# prompts
puesto = "Cajero Polivalente de supermercado"
tareas = """ - Atender a los clientes en la caja registradora.
- Recibir pagos con distintos medios de pago. 
- Escanear y registrar productos. 
- Realizar cierres y arqueos de caja al final del turno. 
- Asistir a los clientes sobre precios, promociones y ubicación de productos. 
- Mantener la caja registradora organizada. 
- Abrir y cerrar la caja registradora con responsabilidad. 
- Verificar y contar la cantidad de billetes en la caja registradora. 
- Verificar precios y promociones vigentes. 
- Gestionar devoluciones y cambios de productos. 
- Proporcionar información sobre productos y servicios de la tienda. 
- Colaborar con el equipo de seguridad para evitar robos. 
- Brindar un servicio amigable y eficiente a los clientes. 
- Manejar reclamos y quejas de los clientes de manera adecuada. 
- Asistir en la capacitación de nuevos empleados. 
- Informar a los supervisores sobre discrepancias en el inventario. """

# prompt system inicial
prompt_system_entrevistas = f"""
Eres un reclutador entrevistando a un candidato para el puesto de {puesto} en Argentina. 
Tu objetivo es hacer preguntas al candidato sobre las tareas del puesto para evaluar su aptitud. 
Las tareas del Cajero Polivalente son: 
{tareas}
Ignora: raza, color, género, religión, nacionalidad. 
"""

# prompts dimensiones
json_dimensiones = '''
[
    {"dimension": "experiencia", "prompt": "Sigue las siguientes instrucciones, reglas estrictas y ejemplos. 
    Instrucciones: 
    - Genera una pregunta directa y comprensible para un adolescente, de menos de 30 palabras, sobre la EXPERIENCIA del candidato en relación a alguna de las tareas mencionadas del Cajero Polivalente; y solicita que ejemplifique.
    Reglas: 
    - La pregunta No debe incluir temas como: atención al cliente, manejo de conflictos y habilidades de comunicación. 
    - Usa una de estas palabras para comenzar la pregunta: qué, cómo, cuándo, por qué, dónde, quién, cuál, cuáles, quiero saber, en qué, de qué manera, hasta qué punto, con qué, qué tan, cómo describirías, qué piensas de. 
    - No uses 'podrías' ni 'puedes' al inicio de la pregunta. 
    - No saludes, no te disculpes, no justifiques, no realices comentarios y no pongas número de pregunta, solo genera la simple pregunta.
    - El formato de salida debe ser solo de esta manera: '¿[pregunta]?' 
    Ejemplo: 
     ¿Qué experiencia tienes en mantener la caja registradora organizada? "},  
    {"dimension": "atención al cliente", "prompt": "Ignora la pregunta anterior, ahora continúa con las siguientes instrucciones, reglas estrictas y ejemplos. 
    Instrucciones: 
    - Genera otra pregunta directa y comprensible para un adolescente, de menos de 30 palabras, sobre la ATENCIÓN AL CLIENTE en relación a alguna tarea del puesto, distinta a la preguntada anteriormente; y solicita que ejemplifique.
    Reglas: 
    - La pregunta no debe ser sobre manejo de conflictos ni habilidades de comunicación. 
    - No uses 'podrías' ni 'puedes' al inicio de la pregunta. 
    - No saludes, no te disculpes, no justifiques, no realices comentarios y no pongas número de pregunta, solo genera la simple pregunta.
    - El formato de salida debe ser solo de esta manera: '¿[pregunta]?' 
    Ejemplo: 
     ¿Cómo respondes a los clientes cuando te solicitan información de algún producto? "},
    {"dimension": "manejo de conflicto", "prompt": "Ignora la pregunta anterior, ahora continúa con las siguientes instrucciones, reglas estrictas y ejemplos. 
    Instrucciones: 
    - Genera otra pregunta simple, directa y comprensible para un adolescente, de menos de 30 palabras, sobre el MANEJO DE CONFLICTOS con algún cliente en el supermercado; y solicita que brinde algún ejemplo. 
    Reglas: 
    - La pregunta no debe ser sobre atención al cliente ni habilidades de comunicación. 
    - No uses 'podrías' ni 'puedes' al inicio de la pregunta. 
    - No saludes, no te disculpes, no justifiques, no realices comentarios y no pongas número de pregunta, solo genera la simple pregunta.
    - El formato de salida debe ser solo de esta manera: '¿[pregunta]?' 
    Ejemplo: 
     ¿Qué estrategias utilizas para resolver alguna situación complicada con un cliente? "},
    {"dimension": "habilidad de comunicación", "prompt": "Ignora la pregunta anterior, ahora continúa con las siguientes instrucciones, reglas estrictas y ejemplos. 
    Instrucciones: 
    - Genera otra pregunta simple, directa y comprensible para un adolescente, de menos de 30 palabras, sobre la HABILIDAD DE COMUNICACIÓN en el supermercado, solicitandóle que brinde algún ejemplo concreto de alguna sobre estas habilidades: claridad en la comunicación, capacidad de expresarse de forma objetiva, clara y respetuosa, o capacidad de interpretar y utilizar señales no verbales. 
    Reglas: 
    - La pregunta no debe ser sobre manejo de conflictos. 
    - No uses 'podrías' ni 'puedes' al inicio de la pregunta. 
    - No saludes, no te disculpes, no justifiques, no realices comentarios y no pongas número de pregunta, solo genera la simple pregunta.
    - El formato de salida debe ser solo de esta manera: '¿[pregunta]?' 
     Ejemplo: 
     ¿Cuáles son tus habilidades de comunicación con los clientes para satisfacer sus necesidades? "}
]
'''
#     - Usa una de estas frases para comenzar, sin repetir la frase anterior: qué, cómo, cuándo, por qué, dónde, quién, cuál, cuáles, quiero saber, en qué, de qué manera, hasta qué punto, con qué, qué tan, cómo describirías, qué piensas de. 
#    - El formato de salida debe ser solo de esta manera: '¿[extrae solo el texto de la pregunta sin modificar su contenido ni agregar comentarios adicionales]?' 

# guardará la conversación entera para la posterior evaluación
dialogo_entrevista = []

# paso a paso donde se guardarán los prompts
messages = []

In [5]:
def generateTokenIBM(myApik):
    
    #curl -X POST 'https://iam.cloud.ibm.com/identity/token' 
    url = "https://iam.cloud.ibm.com/identity/token"
    
    # -d 'grant_type=urn:ibm:params:oauth:grant-type:apikey & apikey=${myApik}
    data = {
        "grant_type": "urn:ibm:params:oauth:grant-type:apikey",
        "apikey": myApik
    }
    
    # -H 'Content-Type: application/x-www-form-urlencoded'
    headers = {"Content-Type": "application/x-www-form-urlencoded"}

    #curl -X POST 'https://iam.cloud.ibm.com/identity/token' -H 'Content-Type: application/x-www-form-urlencoded' 
    # -d 'grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey=${myApik}'
    response = requests.post(url, data=data, headers=headers)  ### La request de CURL.
    access_token = None

    # Verifica si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Parsea la respuesta JSON
        #print("response: ",response,"\n\n")
        json_data = json.loads(response.text)
        
        # Extrae el valor de 'access_token' de la respuesta JSON
        access_token = json_data.get('access_token')
        
        #if access_token:
        #    print("Access Token se genero correctamente")#, access_token)
            # Luego, puedes usar 'access_token' en otra solicitud HTTP, por ejemplo:
            # headers = {"Authorization": f"Bearer {access_token}"}
            # response = requests.get("https://otra-url.com", headers=headers)

        #else:
        #    print("No se pudo extraer el Access Token de la respuesta JSON.")
    else:
        print(f"La solicitud falló con el código de estado {response.status_code}.")
        print(f"Response.text: ")
        json_data = json.loads(response.text)
        print(json_data)
        ## print("all response: ",dir(response))  ## Para ver todo el contenido de response.

    #print("\n","Access Token:", access_token,"\n")
    return access_token
    ##########

In [6]:
# convierte array de prompts a string mixtral
def mixtral_array2string(messages):
    cont_user = 0
    for m in messages:
        if m["role"] == "system":
            input = f"<s> [INST] {m['content']} \n"
        elif m["role"] == "user":
            input += " [INST] " if cont_user > 0 else " "
            input += f"{m['content']} [/INST] "
            cont_user = cont_user + 1
        else:
            input += f"{m['content']}</s> "
    return input

# extrae la pregunta
def extraer_pregunta(cadena):
    try:
        inicio = cadena.index("¿")
        fin = cadena.index("?", inicio) + 1
        return cadena[inicio:fin]
    except ValueError:
        return cadena

# extrae texto json con [] o con {}
def extract_text_between_characters(input_string, characters=""):
    # Utiliza una expresión regular para encontrar todo el texto entre "{...}"
    if characters=="{}":
        pattern = re.compile(r'\{(.*?)\}')
        matches = pattern.findall(input_string)
    elif characters=="[]":
        pattern = re.compile(r'\[(.*?)\]')
        matches = pattern.findall(input_string)
    elif characters=="¿?":
        pattern = re.compile(r'\¿(.*?)\?')
        matches = pattern.findall(input_string)
    else:
        matches = input_string
    return matches

In [7]:
def chat_with_model(messages, model, temperature, max_tokens=250):

    start_time = time.time()
    
    if model == "mistralai/mixtral-8x7b-instruct-v01" or model == "mistralai/mistral-large":
        prompt = mixtral_array2string(messages) # convierte array de prompts a string mixtral
        #print(prompt + "\n")
        url = "https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2023-05-29"
        headers = {
        	"Accept": "application/json",
        	"Content-Type": "application/json",
        	"Authorization": "Bearer " + token
        }
        data = {
            "model_id": model,
            "project_id": project_id,
            #"messages": messages, 
            # en el input debe pasarse un string con todo el prompt, ej: <s>[INST] bla bla [INST]</s>
            # sino tira este error: input must be of type string
            #"input": f"""<s>[INST] {prompt_system_entrevistas} [INST]</s> 
            #[INST] {tmp_prompt} [INST]</s> """, 
            "input": prompt,
            "parameters": {
                "decoding_method": "sample", "temperature": temperature, "top_p": top_p, "top_k": 50, "typical_p": 1
                #"decoding_method": "greedy"
                ,"min_new_tokens": 1
                ,"max_new_tokens": max_tokens
                ,"random_seed": random_seed
                ,"stop_sequences": ["?"]
                ,"repetition_penalty": 1.1
            },
            "moderations": {}
        }
        # ejecuta la api
        response = requests.post(url, headers=headers, json=data)
        if response.status_code != 200:
        	raise Exception("Non-200 response: " + str(response.text))
        response_data = response.json()
        generated_text = response_data['results'][0]['generated_text']
        total_tokens = str(int(response_data['results'][0]['input_token_count']) + int(response_data['results'][0]['generated_token_count']))
    
    else: # gpt
        max_tokens = 4096
        response = openai.chat.completions.create(
            model=model
            ,messages=messages
            ,temperature=temperature
            ,max_tokens=max_tokens
            ,top_p=top_p
            ,frequency_penalty=0.0
            ,presence_penalty=0.0
        )
        generated_text = response.choices[0].message.content
        total_tokens = response.usage.total_tokens

    end_time = time.time()
    # mide tiempo de respuesta
    response_time = end_time - start_time
    
    return generated_text, int(total_tokens), round(response_time, 2)


In [8]:
# Limpia texto de caracteres extendidos y espacios en blanco
def clean_text(texto):
    # Definir un patrón que permita caracteres ASCII imprimibles y caracteres extendidos del español
    caracteres_permitidos = re.compile(r'[^\x20-\x7E\u00A1-\u00FF]')
    texto_limpio = caracteres_permitidos.sub('', texto)
    # Eliminar espacios en blanco innecesarios al principio y al final del texto
    texto_limpio = texto_limpio.strip()
    # Reemplazar más de dos espacios en blanco consecutivos por un solo espacio
    texto_limpio = re.sub(r'\s{2,}', ' ', texto_limpio)
    # Reemplaza basura de json
    return texto_limpio

# convierte una lista a json string
def list_to_json(json_lst):
    json_str = json.dumps(json_lst, ensure_ascii=False) if type(json_lst) == list else json_lst
    json_str = clean_text(json_str)
    return json_str

# convierte un json string a lista
def json_to_list(json_str, characters=""):
    json_str = clean_text(json_str)
    decoded_json_str = json_str.encode().decode('utf-8-sig')
    decoded_json_str = clean_text(decoded_json_str)
    decoded_json_str = decoded_json_str.replace("```json", "")
    decoded_json_str = decoded_json_str.replace("```", "")
    decoded_json_str = decoded_json_str.replace("}[{", "[{")
    decoded_json_str = decoded_json_str.replace("['{", "[{")
    decoded_json_str = decoded_json_str.replace("}']", "}]")
    if characters=="":
        decoded_json_str = extract_text_between_characters(decoded_json_str, characters)
    #print(decoded_json_str)
    try:
        json_lst = json.loads(decoded_json_str)
    except:
        json_lst = decoded_json_str
    return json_lst


In [9]:
# Función para extraer solo la pregunta de un texto
def extract_question_from_text(input):
    prompt = f"""Dado este texto: '{input}', extrae solo la pregunta del texto sin modificar el contenido de la pregunta ni agregar comentarios adicionales. La salida debe ser solo la pregunta extraída. """
    role_message = [{"role": "system", "content": prompt}] 
    response, tokens_used, r_seconds = chat_with_model(role_message, model, temperature, max_tokens)
    return clean_text(response)


# Función para crear el prompt inicial y las preguntas subsecuentes
def create_system_prompt(system_prompt):
    clean_prompt = clean_text(system_prompt) # limpia el texto de caracteres extendidos y espacios en blanco
    system_role_message = [{"role": "system", "content": clean_prompt}]  # El sistema configura el contexto inicial
    return system_role_message


# Función para hacer preguntas al candidato generadas por el modelo
def generate_question(messages, prompt):
    global total_tokens_used, response_seconds
    messages.append({"role": "user", "content": prompt})  # El asistente genera una pregunta
    # ejecuta el modelo
    response, total_tokens_used, response_seconds = chat_with_model(messages, model, temperature, max_tokens)
    question = clean_text(response)
    #question = extract_question_from_text(response) 
    #question = extraer_pregunta(question)
    messages.append({"role": "assistant", "content": question})  # El asistente hace la pregunta generada
    return question


# Función para evaluar si se necesita aclaración
def evaluate_response(messages, question, answer):
    global total_tokens_used, response_seconds
    clean_answer = answer.replace("|", " ") # quita | si hay en la respuesta

    # nuevo user prompt #
    prompt = f"""Respuesta del candidato: '{clean_answer}'. En base a la respuesta del candidato, tú como entrevistador sigue las siguientes instrucciones. 
Instrucciones: 
 - Si el candidato no comprende la pregunta: reformula la pregunta anterior de forma más comprensible y más directa, no agregues comentarios adicionales, solo genera la simple pregunta. 
 - Si la respuesta es incompleta, o no brinda detalles claros, o es solo una afirmación o negación sin detalles ni ejemplos: genera otra pregunta con lenguaje comprensible y directo para que el candidato brinde mayor información sobre su respuesta anterior, no agregues comentarios adicionales, solo genera la simple pregunta. 
 - Si la respuesta expresa no saber o no tener experiencia en el área: responde solamente "(No sabe)". 
 - Si la respuesta es clara y brinda detalles: responde solamente "(Responde bien)". 
"""

    # agrega el user prompt
    messages.append({"role": "user", "content": clean_text(prompt)})
    # ejecuta el modelo
    response, total_tokens_used, response_seconds = chat_with_model(messages,model, temperature, max_tokens)
    # las opciones que no necesita repreguntar
    clarification_needed = not (any(word in response.lower() for word in ['no sabe', 'responde bien']))
    
    #if clarification_needed: # depura la pregunta
    #    response = extraer_pregunta(question)
    #    response = extract_question_from_text(response) 
    
    # El asistente devuelve la repregunta
    messages.append({"role": "assistant", "content": clean_text(response)})

    #v_response = response.split("|")
    # devuelve si necesita clarificar y la respuesta del asistente
    
    #return clarification_needed, v_response[len(v_response)-1]
    return clarification_needed, clean_text(response)

In [10]:
# Función para la evaluacion de dimensiones
def evaluate_dimensions(json_dialogue):
    global total_tokens_used, messages, response_seconds
    json_str = list_to_json(json_dialogue)

    sys_prompt = f"""Eres un reclutador y debes evaluar la idoneidad de un candidato para el puesto de {puesto}. Las tareas del puesto son: 
    {tareas}. 
    Ignora: raza, color, género, religión, nacionalidad."""

    usr_prompt = f"""[Basándote en la siguiente conversación en formato JSON entre el entrevistador y el candidato:
[Inicio de conversación]
{json_str}
[Fin de conversación]
Debes evaluar la idoneidad de este candidato para el puesto de {puesto} considerando las dimensiones mencionadas en la conversación de la siguiente manera:

A) Genera una sola respuesta en formato JSON de la siguiente manera:
[{{"Dimension": "[dimensión mencionada en la conversación]", "Calificacion": [calificación], "Evaluacion": "[resumen de la evaluación]"}}]

B) Completa el valor del atributo "Calificación" del JSON anterior con una calificación muy exigente de 1 a 5, basada en los siguientes criterios:
  1 = Pésimo candidato: No cumple con los requisitos básicos del puesto, sin ejemplos concretos, muestra falta total de experiencia y habilidades para la dimensión en cuestión. No se sabe expresar, comete muchas faltas de ortografía o no responde a la pregunta realizada sobre la dimensión.
  2 = Deficiente candidato: Cumple con algunos requisitos mínimos pero tiene varias áreas de mejora significativas en la dimensión en cuestión. Su respuesta es incompleta y no responde con asertividad o de forma directa a la pregunta realizada sobre la dimensión.
  3 = Candidato aceptable: Cumple con la mayoría de los requisitos, tiene habilidades adecuadas en la dimensión en cuestión, pero puede mejorar en algunos aspectos. Su respuesta no precisa detalles sobre la dimensión y parece exagerada.
  4 = Buen candidato: Cumple y supera la mayoría de los requisitos, demuestra habilidades sólidas en la dimensión en cuestión, con pocas áreas de mejora. Su respuesta es clara y brinda ejemplos concretos sobre la dimensión.
  5 = Excelente candidato: Cumple todos los requisitos, demuestra habilidades excelentemente calificadas y mucha capacidad en la dimensión en cuestión, sin áreas de mejora significativas. Su respuesta es sobresaliente, brinda ejemplos bien concretos relacionados a la pregunta realizada, sin divagar ni alucinar sobre la dimensión, demostrando su gran capacidad.
 La pésima ortografía es motivo de calificación máxima 2.
 Respuestas muy buenas pero divagantes deben tener una calificación máxima de 3.
 Respuestas breves de menos de 10 palabras que contengan palabras iguales o similares a: 'excelente', 'muy bueno', 'mucha experiencia', o solo una afirmación sin detalles es una calificación de 1.
 Respuestas vacías o que no responden a la pregunta es una calificación de 1.
C) Completa el valor del atributo "Evaluacion" del JSON anterior con un resumen de evaluación del candidato en menos de 60 palabras, resaltando aspectos positivos y negativos que consideres.
D) El formato de salida de tu respuesta debe ser únicamente en el formato JSON anterior. No respondas en otro formato ni hagas comentarios adicionales por fuera de ese formato JSON.
"""
    
    messages = [{"role": "system", "content": clean_text(sys_prompt)}]  # El sistema configura el contexto inicial
    messages.append({"role": "user", "content": clean_text(usr_prompt)}) # Genera el user prompt
    # ejecuta el modelo
    response, total_tokens_used, response_seconds = chat_with_model(messages, model, temperature, max_tokens)
    messages.append({"role": "assistant", "content": clean_text(response)}) # El asistente devuelve la respuesta
    #print("evaluate_dimensions + response = "+response+"\n")
    # convierte respuesta json en lista
    json_response = json_to_list(response, "[]")
    # extrae solo json
    #json_response = extract_text_between_characters(json_response, characters="[]")
    
    return json_response, total_tokens_used


In [11]:
# Función para la evaluacion final
def evaluate_final(json_evaluation):
    global total_tokens_used, messages, response_seconds
    json_str = list_to_json(json_evaluation)
    sys_prompt = f"""Eres un reclutador y debes evaluar la idoneidad de un candidato para el puesto de {puesto}. 
    Ignora: raza, color, género, religión, nacionalidad."""
    usr_prompt = f"""Basándote en la siguiente evaluación del candidato en formato JSON que se realizó previamente: 
        [Inicio de evaluación]
        {json_str}
        [Fin de evaluación]
        Debes generar la respuesta únicamente en formato JSON de la siguiente manera: 
        {{"Dimension": "Evaluación Final", "Calificacion": [Obtener el promedio de todas las Calificaciones, solo el número resultante.], 
        "Evaluacion": "[Generar un resumen general de las evaluaciones de todas las dimensiones mencionadas en menos de 100 palabras, 
        incluyendo aspectos positivos y negativos para evaluar la idoneidad de este candidato para el puesto de {puesto}. No menciones 
        el número de calificación, solo genera el resumen.]"}}"""
    # El sistema configura el contexto inicial
    messages = [{"role": "system", "content": clean_text(sys_prompt)}] 
    # Genera el user prompt
    messages.append({"role": "user", "content": clean_text(usr_prompt)})
    # ejecuta el modelo
    response, total_tokens_used, response_seconds = chat_with_model(messages, model, temperature, max_tokens)
    messages.append({"role": "assistant", "content": clean_text(response)}) # El asistente devuelve la respuesta
    #print("evaluate_final + response = "+response+"\n")
    # convierte respuesta json en lista
    json_response = json_to_list(response, "{}")
    # extrae solo json
    #json_response = extract_text_between_characters(json_response, characters="{}")
    
    return json_response, total_tokens_used


In [12]:
def interview():
    global dialogo_entrevista, messages, total_response_seconds, avg_response_seconds, token
    iteraciones = 0
    
    # crea el system prompt
    messages = create_system_prompt(prompt_system_entrevistas)
    
    # Cargar json dimensiones en una lista
    data_dimensiones = json_to_list(json_dimensiones)

    #genera token
    token = generateTokenIBM(api_key)

    # rondas de preguntas por dimensión
    for item in data_dimensiones:
        dimension = item['dimension']
        prompt_dimension = item['prompt']
        prompt_dimension = clean_text(prompt_dimension) # limpia textos
        
        # genera pregunta
        question = generate_question(messages, prompt_dimension)
        question_extracted = extraer_pregunta(question) # extrae solo la pregunta x si hay algún comentario
        print(f"\nPregunta del entrevistador: {question_extracted}")

        total_response_seconds = total_response_seconds + response_seconds
        iteraciones = iteraciones + 1
        
        # Respuesta del candidato
        answer = input("Respuesta del candidato: ")
        answer = clean_text(answer) # limpia textos
        
        # Log conversacion
        dialogo_entrevista.append({"dimension": dimension, "Pregunta del entrevistador": question_extracted, "Respuesta del candidato": answer})
        
        # Repregunta según respuesta
        b_repregunta, s_repregunta = evaluate_response(messages, question, answer)
        
        # si tiene que repreguntar
        if b_repregunta:
            # Pregunta de seguimiento
            requestion_extracted = extraer_pregunta(s_repregunta) # extrae solo la pregunta x si hay algún comentario
            print(f"Repregunta del entrevistador: {requestion_extracted}")
            # Respuesta del candidato
            answer = input("Respuesta del candidato: ")
            answer = clean_text(answer) # limpia textos
            # Log conversacion
            dialogo_entrevista.append({"dimension": dimension, "Pregunta del entrevistador": requestion_extracted, "Respuesta del candidato": answer})

    avg_response_seconds = round((total_response_seconds / iteraciones), 2)

    return messages

## ENTREVISTA

In [13]:
resultado = interview()


Pregunta del entrevistador: ¿Cuál ha sido tu experiencia en realizar cierres y arqueos de caja al final del turno?


Respuesta del candidato:  buena experiencia


Repregunta del entrevistador: ¿Podrías contarme sobre una situación en la que hayas realizado un cierre y arqueo de caja al final de tu turno?


Respuesta del candidato:  todos los días hacía el arqueo de caja ante cambios de turno contabilizando los billetes y llenando la planilla de informes de caja



Pregunta del entrevistador: ¿Cómo gestionaste alguna vez una devolución o cambio de producto solicitado por un cliente?


...

Respuesta del candidato:  la persona estaba alcoholizada y agresiva, lo invité a retirarse y llamé a seguridad



Pregunta del entrevistador: ¿Podrías darme un ejemplo de cómo has empleado habilidades de comunicación clara y precisa con un cliente al explicarle sobre la ubicación de un producto o las promociones vigentes?


Respuesta del candidato:  explico al cliente donde están los productos en góndola y si hay una promoción se la hago saber


Repregunta del entrevistador: ¿Podrías darnos un caso práctico en el que hayas detectado preocupaciones o necesidades de un cliente incluso antes de que este verbalizara dichas demandas?


Respuesta del candidato:  a veces el cliente se acerca a la caja para preguntar y yo les indico, otras veces si les noto cara de preocupación, les pregunto cual es su inquietud en forma amigable


In [14]:
print(f"total_tokens_used: {total_tokens_used} - avg_response_seconds: {avg_response_seconds}")

total_tokens_used: 3072 - avg_response_seconds: 6.44


In [15]:
prompt = mixtral_array2string(messages) # convierte array de prompts a string mixtral
prompt

'<s> [INST] Eres un reclutador entrevistando a un candidato para el puesto de Cajero Polivalente de supermercado en Argentina. Tu objetivo es hacer preguntas al candidato sobre las tareas del puesto para evaluar su aptitud. Las tareas del Cajero Polivalente son: - Atender a los clientes en la caja registradora.- Recibir pagos con distintos medios de pago. - Escanear y registrar productos. - Realizar cierres y arqueos de caja al final del turno. - Asistir a los clientes sobre precios, promociones y ubicación de productos. - Mantener la caja registradora organizada. - Abrir y cerrar la caja registradora con responsabilidad. - Verificar y contar la cantidad de billetes en la caja registradora. - Verificar precios y promociones vigentes. - Gestionar devoluciones y cambios de productos. - Proporcionar información sobre productos y servicios de la tienda. - Colaborar con el equipo de seguridad para evitar robos. - Brindar un servicio amigable y eficiente a los clientes. - Manejar reclamos y 

In [16]:
#resultado

In [17]:
dialogo_entrevista

[{'dimension': 'experiencia',
  'Pregunta del entrevistador': '¿Cuál ha sido tu experiencia en realizar cierres y arqueos de caja al final del turno?',
  'Respuesta del candidato': 'buena experiencia'},
 {'dimension': 'experiencia',
  'Pregunta del entrevistador': '¿Podrías contarme sobre una situación en la que hayas realizado un cierre y arqueo de caja al final de tu turno?',
  'Respuesta del candidato': 'todos los días hacía el arqueo de caja ante cambios de turno contabilizando los billetes y llenando la planilla de informes de caja'},
 {'dimension': 'atención al cliente',
  'Pregunta del entrevistador': '¿Cómo gestionaste alguna vez una devolución o cambio de producto solicitado por un cliente?',
  'Respuesta del candidato': 'verificaba que el estado del producto no haya sido modificado y luego si estaba ok lo registraba en la caja como cambio de producto'},
 {'dimension': 'atención al cliente',
  'Pregunta del entrevistador': '¿Cómo abordas las consultas de los clientes sobre pre

## EVALUACION X DIMENSION

In [18]:
# Evalúa las dimensiones
max_tokens = 4096
evaluacion_dimensiones, total_tokens_used = evaluate_dimensions(dialogo_entrevista)
evaluacion_dimensiones

[{'Dimension': 'experiencia',
  'Calificacion': 4,
  'Evaluacion': 'El candidato demuestra tener una buena experiencia en realizar cierres y arqueos de caja, proporciona ejemplos concretos y específicos de su experiencia previa en esta área.'},
 {'Dimension': 'atención al cliente',
  'Calificacion': 3,
  'Evaluacion': 'El candidato tiene habilidades adecuadas en atención al cliente, aunque podría mejorar en proporcionar más detalles sobre cómo gestiona las consultas de los clientes sobre precios y promociones.'},
 {'Dimension': 'manejo de conflicto',
  'Calificacion': 2,
  'Evaluacion': 'El candidato ha tenido experiencias en el manejo de conflictos con clientes, pero su respuesta carece de detalles sobre cómo abordó la situación y cómo la resolvió.'},
 {'Dimension': 'habilidad de comunicación',
  'Calificacion': 4,
  'Evaluacion': 'El candidato demuestra habilidades sólidas en comunicación clara y precisa, proporciona ejemplos concretos y relevantes sobre cómo emplea sus habilidades d

In [19]:
print(f"total_tokens_used: {total_tokens_used}")

total_tokens_used: 2297


In [20]:
#prompt = mixtral_array2string(messages) # convierte array de prompts a string mixtral
#prompt

## EVALUACION FINAL

In [21]:
# Evaluación final
max_tokens = 4096
evaluacion_final, total_tokens_used = evaluate_final(evaluacion_dimensiones)
evaluacion_final

{'Dimension': 'Evaluación Final',
 'Calificacion': 3.5,
 'Evaluacion': 'El candidato tiene una buena experiencia en cierres y arqueos de caja, demostrando habilidades sólidas en comunicación. Sin embargo, hay margen de mejora en la descripción de cómo gestiona consultas de clientes sobre precios y promociones, así como en la provisión de detalles sobre el manejo de conflictos.'}

In [22]:
print(f"total_tokens_used: {total_tokens_used}")

total_tokens_used: 690


In [23]:
#prompt = mixtral_array2string(messages) # convierte array de prompts a string mixtral
#prompt

In [24]:
#messages

In [25]:
#genera token
#token = generateTokenIBM(api_key)